# Deep Learning Model

## Imports

In [ ]:
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import re
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from sacrebleu import corpus_bleu
import pandas as pd
import numpy as np
import pyarabic.araby as araby
import pyarabic.number as number
import pyarabic.named as named
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Loading Dataset

In [ ]:
df_total=pd.read_csv("totalDatasetWithEmo2.csv",encoding="utf-8-sig")
df_totalTokens=pd.read_csv("totalDatasetTokensWithEmo2.csv",encoding="utf-8-sig")

## Splitting dataset

In [ ]:
df_to_train,df_to_test=train_test_split(df_totalTokens, test_size=0.1)
traincontextAndResponse=df_to_train["context"]
traincontextAndResponseLabels=df_to_train["emotion2"]

## Setting hyperparameters

In [ ]:
vocab_size = 12900
embedding_dim = 16
max_len = 25
oov_token = "<OOV>"

## tokenizing and encoding class labels

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(traincontextAndResponse)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(traincontextAndResponse.to_list())
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
trainingLabels=traincontextAndResponseLabels.to_list()
lbl_encoder = LabelEncoder()
lbl_encoder.fit(traincontextAndResponseLabels)
trainingLabels = lbl_encoder.transform(trainingLabels)
labels=set(trainingLabels)
num_classes = len(labels)
num_classes

## Creating model architecture

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='SGD', metrics=['accuracy'])

model.summary()

## Training

In [ ]:
epochs = 500
history = model.fit(padded_sequences, np.array(trainingLabels), epochs=epochs)

## Evaluating

In [ ]:
testcontextAndResponse=df_to_test["context"]
testcontextAndResponseLabels=df_to_test["emotion2"]

testsequences = tokenizer.texts_to_sequences(testcontextAndResponse.to_list())
testpadded_sequences = pad_sequences(testsequences, truncating='post', maxlen=max_len)

testLabels = lbl_encoder.transform(testcontextAndResponseLabels)
labels=set(testLabels)
num_classes = len(labels)

In [ ]:
results = model.evaluate(testpadded_sequences, np.array(testLabels))

In [ ]:
model.metrics_names

In [ ]:
results

## Saving Model

In [ ]:

# to save the trained model
model.save("chat_model5")

import pickle

# to save the fitted tokenizer
with open('tokenizer5.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder5.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)